# Module 1: Introduction to Exploratory Analysis 

What we'll be doing in this notebook:
-----

 1.  Checking variable type
 2.  Checking for missing variables 
 3.  Number of observations in the dataset
 4.  Descriptive statistics

### Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser

# The command below means that the output of multiple commands in a cell will be output at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

# Show figures in notebook
%matplotlib inline

### Import dataset

We read in our dataset

In [2]:
kiva_locations = pd.read_csv("/home/catherine/Desktop/DataScience1/data/Kenya/kiva_ke_locations.csv")
kiva_loans_ke = pd.read_csv("/home/catherine/Desktop/DataScience1/data/Kenya/kiva_loans_ke.csv")
kiva_loan_themes = pd.read_csv("/home/catherine/Desktop/DataScience1/data/Kenya/loan_theme_ke.csv")

In the cell below, we take a random sample of 2 rows to get a feel for the data.

In [3]:
kiva_loans_ke.sample(n = 1002)

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
53852,1123457,525,525,General Store,Retail,"to buy more stock of bread, flour, soap, sugar...",KE,Kenya,Machakos,KES,138.0,2016-07-26 06:33:24+00:00,2016-07-22 07:00:00+00:00,2016-09-02 22:24:53+00:00,14,13,"#Parent, #Woman Owned Biz, #Repeat Borrower",female,monthly,2016-07-26
20296,823856,225,225,Milk Sales,Food,to buy more stock of milk to sell,KE,Kenya,Kapsabet,KES,133.0,2015-01-05 09:31:06+00:00,2014-12-15 08:00:00+00:00,2015-01-06 19:46:34+00:00,8,1,NaN,female,monthly,2015-01-05
75715,1334887,0,200,Farming,Agriculture,to start an agro-vet business (a shop for farm...,KE,Kenya,Thika,KES,156.0,2017-07-06 09:15:02+00:00,2017-06-29 07:00:00+00:00,NaN,14,0,"#Animals, #Parent",female,monthly,2017-07-06
63732,1218679,50,50,Home Energy,Personal Use,to buy an eco-friendly jiko or stove.,KE,Kenya,Eldoret,KES,156.0,2017-01-11 11:21:33+00:00,2016-12-28 08:00:00+00:00,2017-01-11 17:28:15+00:00,8,1,"user_favorite, #Eco-friendly, #Health and Sani...",female,monthly,2017-01-11
48854,1054067,700,700,Farming,Agriculture,to purchase hybrid seeds and fertilizer for th...,KE,Kenya,Chwele,KES,202.0,2016-04-14 06:16:12+00:00,2016-04-02 07:00:00+00:00,2016-04-25 03:11:45+00:00,9,26,"#Eco-friendly, #Sustainable Ag, #Technology, #...","female, female, female, female, female, female...",bullet,2016-04-14
16077,1085312,125,125,Clothing,Clothing,Buy stock for my mtumba business,KE,Kenya,NaN,KES,NaN,2014-11-05 11:34:48+00:00,2015-01-06 21:31:03+00:00,2014-12-27 01:54:33+00:00,3,9,NaN,female,monthly,2014-11-05
44292,1018248,600,600,Cereals,Food,to buy cereals.,KE,Kenya,Nandi Hills,KES,156.0,2016-02-04 08:34:25+00:00,2016-02-07 08:00:00+00:00,2016-02-29 04:44:42+00:00,14,5,"#Parent, #Woman Owned Biz",female,monthly,2016-02-04
27788,876468,275,275,Farming,Agriculture,to buy certified farm inputs for more production.,KE,Kenya,Kisii,KES,133.0,2015-04-27 09:08:33+00:00,2015-03-31 07:00:00+00:00,2015-05-28 20:29:22+00:00,12,8,"#Elderly, #First Loan, #Supporting Family",female,monthly,2015-04-27
46706,1036911,300,300,Services,Services,to add float to use in her business.,KE,Kenya,Msambweni,KES,164.0,2016-03-11 06:07:25+00:00,2016-03-11 08:00:00+00:00,2016-03-19 15:01:32+00:00,13,12,"#Single, #Technology",female,irregular,2016-03-11
48858,1054174,450,450,Farming,Agriculture,to purchase hybrid seeds and fertilizer to imp...,KE,Kenya,Chwele,KES,202.0,2016-04-14 07:58:30+00:00,2016-04-02 07:00:00+00:00,2016-04-22 23:22:27+00:00,9,18,"#Eco-friendly, #Sustainable Ag, #Technology, #...","male, male, female, female, female, female",bullet,2016-04-14


### 1) Type Checking
<a id='type_check'></a>

Type is very important in Python programing, because it affects the types of functions you can apply to a series. There are a few different types of data you will see regularly (see [this](https://en.wikibooks.org/wiki/Python_Programming/Data_Types) link for more detail):
* **int** - a number with no decimal places. example: loan_amount field
* **float** - a number with decimal places. example: partner_id field
* **str** - str is short for string. This type formally defined as a sequence of unicode characters. More simply, string means that the data is treated as a word, not a number. example: sector
* **boolean** - can only be True or False. There is not currently an example in the data, but we will be creating a gender field shortly.
* **datetime** - values meant to hold time data. Example: posted_date

Let's check the type of our variables using the examples we saw in the cell above.

In [5]:
# Here are all of the columns
kiva_loans_ke.columns.tolist()

['id',
 'funded_amount',
 'loan_amount',
 'activity',
 'sector',
 'use',
 'country_code',
 'country',
 'region',
 'currency',
 'partner_id',
 'posted_time',
 'disbursed_time',
 'funded_time',
 'term_in_months',
 'lender_count',
 'tags',
 'borrower_genders',
 'repayment_interval',
 'date']

In [56]:
# Find the dtype, aka datatype, for a column
df['id_number'].dtype

dtype('int64')

In [57]:
# Try this - Pick a couple of columns and check their type on your own


### 2) Do I have missing values?

<a id='missing_check'></a>

Missing data in the training data set can reduce the power / fit of a model or can lead to a biased model because we have not analysed the behavior and relationship with other variables correctly. It can lead to wrong prediction or classification.

If we have missing data, is the missing data at random or not? If data is missing at random, the data distribution is still representative of the population. You can probably ignore the missing values as an inconvenience. However, if the data is systematically missing, the analysis you do may be biased. You should carefully consider the best way to clean the data, it may involve dropping some data.

We want to see how many values are missing in certain variable columns. One way to do this is to count the number of null observations. 

For this, we wrote a short function to apply to the dataframe. 

We print out the first few observations, but you can remove the .head() to print out all columns. 

In [7]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
## Check how many are missing by column, and then check which ones have any missing values
print(kiva_loans_ke.apply(num_missing, axis=0).where(lambda x : x != 0).dropna()) 
#axis=0 defines that function is to be applied on each column

Missing values per column:
use                   713.0
region               8752.0
partner_id           8372.0
disbursed_time        381.0
funded_time          5446.0
tags                23186.0
borrower_genders      712.0
dtype: float64


In [59]:
#checking the % of missing values
total = df.isnull().sum().sort_values(ascending = False)

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)

missing_kiva_loans_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_kiva_loans_data

,Total,Percent
funded_date,937,0.155674
use,342,0.056820
description,342,0.056820
location_country_code,17,0.002824
sector,0,0.000000
repayment_term,0,0.000000
funded_amount,0,0.000000
status,0,0.000000
lender_count,0,0.000000
loan_amount,0,0.000000


### 3) Sanity Checks
<a id='obs_check'></a>

**Does the dataset match what you expected to find?**
- is the range of values what you would expect. For example, are all loan_amounts above 0.
- do you have the number of rows you would expect
- is your data for the date range what you would expect. For example, is there a strange year in the data like 1880.
- are there unexpected spikes when you plot the data over time


In the command below we find out the number of rows and number of columns by using the function shape. You can also use len(df.index) to find the number of rows.

In [9]:
print(f'There are {kiva_loans_ke.shape[0]} observations and {kiva_loans_ke.shape[1]} features')

There are 75825 observations and 20 features


Remember, each row is an observation and each column is a potential feature. 

Remember we need a large amount of data for machine learning.

### 4) Descriptive statistics of the dataset

<a id='desc_stats'></a>

In [61]:
#Try out - Write code that provides key summary statistics for each numeric column.

In order to get the same summary statistics for categorical columns (string) we need to do a little data wrangling. The first line of code filters for all columns that are a data type object. As we know from before this means they are considered to be a string. The final row of code provides summary statistics for these character fields.

In [62]:
categorical = df.dtypes[df.dtypes == "object"].index
df.describe()

,status,funded_date,location_country_code,sector,description,use
count,6019,5082,6002,6019,5677,5677
unique,3,4453,30,14,5277,4325
top,funded,2018-07-22T15:54:41Z,TZ,Food,With the income produced during the previous l...,to pay for a stove.
freq,5082,9,400,1738,2,80


In [63]:
#Try out - What's the other way one can obtain the information above?

In the table above, there are 4 really useful fields: 

1) **count** - total number of fields populated (Not empty). 

2) **unique** - tells us how many different unique ways this field is populated.

3) **top** - tells us the most popular data point.

4) **freq** - tells us that how frequent the most popular category is in our dataset. 

In [12]:
kiva_loans_ke['sector'].unique()

array(['Services', 'Food', 'Retail', 'Clothing', 'Agriculture',
       'Personal Use', 'Arts', 'Transportation', 'Housing', 'Construction',
       'Education', 'Manufacturing', 'Health', 'Entertainment', 'Wholesale'], dtype=object)

What is next
-----

In the next section, we move on to exploratory data analysis (EDA).

<br>
<br> 
<br>

----